In [33]:
import sys

import pandas as pd

sys.path.append("/home/jovyan/work/sem-covid/")
sys.path = list(set(sys.path))
import os

os.getcwd()
os.chdir('/home/jovyan/work/sem-covid/')
from sem_covid import config
import json
import numpy as np

from sem_covid.services.store_registry import StoreRegistry, store_registry


In [34]:
es_store = store_registry.es_index_store()

In [35]:
#Load dataframes from elastic
pwdb_df=es_store.get_dataframe(index_name=config.PWDB_ELASTIC_SEARCH_INDEX_NAME)
eu_cellar_df = es_store.get_dataframe(index_name=config.EU_CELLAR_ELASTIC_SEARCH_INDEX_NAME)
eu_timeline_df = es_store.get_dataframe(index_name=config.EU_TIMELINE_ELASTIC_SEARCH_INDEX_NAME)
ir_timeline_df = es_store.get_dataframe(index_name=config.IRELAND_TIMELINE_ELASTIC_SEARCH_INDEX_NAME)


100% (1288 of 1288) |####################| Elapsed Time: 0:00:00 Time:  0:00:00
100% (2818 of 2818) |####################| Elapsed Time: 0:00:00 Time:  0:00:00
100% (210 of 210) |######################| Elapsed Time: 0:00:00 Time:  0:00:00
100% (1859 of 1859) |####################| Elapsed Time: 0:00:00 Time:  0:00:00


In [36]:
# Print dataframe columns
pwdb_df.columns


Index(['identifier', 'title', 'title_national_language', 'country',
       'start_date', 'end_date', 'date_type', 'type_of_measure',
       'status_of_regulation', 'category', 'subcategory', 'creation_date',
       'background_info_description', 'content_of_measure_description',
       'use_of_measure_description', 'actors', 'target_groups', 'funding',
       'involvement_of_social_partners_description',
       'social_partner_involvement_form', 'social_partner_role',
       'is_sector_specific', 'private_or_public_sector',
       'is_occupation_specific', 'sectors', 'occupations', 'sources'],
      dtype='object')

In [37]:
# What are the textual columns in PWDB dataset
textual_colums = [col for col in pwdb_df.columns if "description" in col]
print(textual_colums)

['background_info_description', 'content_of_measure_description', 'use_of_measure_description', 'involvement_of_social_partners_description']


In [58]:
# Print only textual columns
pwdb_df[textual_colums]


,background_info_description,content_of_measure_description,use_of_measure_description,involvement_of_social_partners_description
_id,,,,
adc5c75937bc7f7198f534d08b85bd50c9521bfd3f319a090932b5d0bae54de0,"During the COVID-19 crisis, teleworking has be...",Applied for a period of three years and coveri...,All employees in the private sector (with the ...,The agreement is a social partner initiative w...
2372d71eb9ad6e6a70982e02bbe802db004ed49d91b2264c0a2e8e41571002cc,"In the face of the Corona crisis, employers ha...",The COVID-19 risk group regulation lists the m...,According to information from the Minister fro...,The expert group which elaborated the definiti...
8735e268191e9e5cbd3d2a44ca53d297e31746b5f1e24b941db6225a25848353,The Council of Ministers has agreed to authori...,"In total, €136.3 million will be allocated to ...",No data available.,No involvement reported
18bcd22116c46919e03a3345f793c3859855227ac942e69dd13cbfcd588e1044,"The Act 136/2020 Coll., selected modifications...",Eligible are all self-employed persons perform...,Not available yet.,"Social partners, who are members of the tripar..."
b94d8aa95fbdeb1bb832b01fbe5d6e9bf9fc36fceb14f7ba370a963f472fe35b,After the Financial Shield 1.0 was impelemente...,The main characteristics of the measure are as...,Not known yet.,None.
...,...,...,...,...
cb014a456b14c3621dd318a12e611f70c2a9636be9fe181072bd4bf5917a40fa,During the period of validity of the extraordi...,"Through this measure, the social security syst...",There are no estimates as to the number of wor...,"Social partners were only consulted, but not i..."
d233b17dc2b98f14269c2b22be78d93ec5ccf2a0013b86f09175c69353c5800b,The Dutch government presented a package of em...,This particular measure aims to provide extra ...,No information available yet.,The main peak level social partners have been ...
77d7e3c52aaf78bdfb1a1667641db1293bbff862440c547fc3f6e8ab4fbd0d4e,The case was initiated by the Dutch government...,Enterprises (large and SMEs) need to offer gua...,No information available.,"The main, national, level social partners have..."


In [55]:
#Merge content of the textual columns into one new column
pwdb_df["merged_content"] = pwdb_df[textual_colums].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

"During the COVID-19 crisis, teleworking has been identified as a vital pillar to keep companies working and prevent social hardship. Discussions between the two representative social partners OGBL and LCGB and the employer association UEL, together with the Ministry of Work and Employment, led in a first instance to a joint assessment on teleworking at the level of the Economic and Social Council CES. From there, discussions continued between the social partners and an inter-professional agreement could have been signed between the social partners in October 2020.  Applied for a period of three years and covering all sectors in Luxembourg (with the exception of transport), the agreement provides a definition of teleworking: \r\r* Teleworking is identified as a form of organisational work, conducted by digital means and usually done at the company, but that is transferred to the location where the employee lives. \r* The work is considered as teleworking when it is applied to occasiona

In [57]:
#What are the columns that shows dates
date_columns = [col for col in pwdb_df.columns if "date" in col]
pwdb_df[date_columns]

,start_date,end_date,date_type,creation_date
_id,,,,
adc5c75937bc7f7198f534d08b85bd50c9521bfd3f319a090932b5d0bae54de0,10/20/2020,None,Open ended,11/02/2020
2372d71eb9ad6e6a70982e02bbe802db004ed49d91b2264c0a2e8e41571002cc,05/06/2020,05/31/2021,Temporary,07/09/2020
8735e268191e9e5cbd3d2a44ca53d297e31746b5f1e24b941db6225a25848353,08/25/2020,None,Open ended,10/24/2020
18bcd22116c46919e03a3345f793c3859855227ac942e69dd13cbfcd588e1044,03/01/2020,08/31/2020,Temporary,04/06/2020
b94d8aa95fbdeb1bb832b01fbe5d6e9bf9fc36fceb14f7ba370a963f472fe35b,01/01/2021,None,Temporary,12/31/2020
...,...,...,...,...
cb014a456b14c3621dd318a12e611f70c2a9636be9fe181072bd4bf5917a40fa,03/18/2020,01/31/2021,Temporary,04/13/2020
d233b17dc2b98f14269c2b22be78d93ec5ccf2a0013b86f09175c69353c5800b,05/20/2020,None,Temporary,07/13/2020
77d7e3c52aaf78bdfb1a1667641db1293bbff862440c547fc3f6e8ab4fbd0d4e,03/17/2020,None,Temporary,04/21/2020


In [59]:
# Create new dataframe with the wanted content and columns from PWDB dataset
tmp_pwdb_df = pd.DataFrame(pwdb_df[["creation_date","title","merged_content"]])
tmp_pwdb_df["doc_source"]="pwdb"


In [60]:
# See the newly created PWDB dataframe
tmp_pwdb_df

,creation_date,title,merged_content,doc_source
_id,,,,
adc5c75937bc7f7198f534d08b85bd50c9521bfd3f319a090932b5d0bae54de0,11/02/2020,Agreement on a teleworking regime,"During the COVID-19 crisis, teleworking has be...",pwdb
2372d71eb9ad6e6a70982e02bbe802db004ed49d91b2264c0a2e8e41571002cc,07/09/2020,Special protection for COVID-19 risk groups at...,"In the face of the Corona crisis, employers ha...",pwdb
8735e268191e9e5cbd3d2a44ca53d297e31746b5f1e24b941db6225a25848353,10/24/2020,Funds for innovative renewable projects in And...,The Council of Ministers has agreed to authori...,pwdb
18bcd22116c46919e03a3345f793c3859855227ac942e69dd13cbfcd588e1044,04/06/2020,Waiver of advance payments for social and heal...,"The Act 136/2020 Coll., selected modifications...",pwdb
b94d8aa95fbdeb1bb832b01fbe5d6e9bf9fc36fceb14f7ba370a963f472fe35b,12/31/2020,Financial Shield 2.0: Small and medium-sized e...,After the Financial Shield 1.0 was impelemente...,pwdb
...,...,...,...,...
cb014a456b14c3621dd318a12e611f70c2a9636be9fe181072bd4bf5917a40fa,04/13/2020,Automatic extension of unemployment benefits,During the period of validity of the extraordi...,pwdb
d233b17dc2b98f14269c2b22be78d93ec5ccf2a0013b86f09175c69353c5800b,07/13/2020,Extra subsidies to institutions in the cultura...,The Dutch government presented a package of em...,pwdb
77d7e3c52aaf78bdfb1a1667641db1293bbff862440c547fc3f6e8ab4fbd0d4e,04/21/2020,Expanding business loan guarantee scheme,The case was initiated by the Dutch government...,pwdb


In [39]:
# Print dataframe columns
eu_cellar_df.columns

Index(['work', 'title', 'cdm_types', 'cdm_type_labels', 'resource_types',
       'resource_type_labels', 'eurovoc_concepts', 'eurovoc_concept_labels',
       'subject_matters', 'subject_matter_labels', 'directory_codes',
       'directory_codes_labels', 'celex_numbers', 'legal_elis', 'id_documents',
       'same_as_uris', 'authors', 'author_labels', 'full_ojs', 'oj_sectors',
       'internal_comments', 'is_in_force', 'dates_document', 'dates_created',
       'legal_dates_entry_into_force', 'legal_dates_signature', 'manifs_pdf',
       'manifs_html', 'pdfs_to_download', 'htmls_to_download', 'dossiers',
       'related_works', 'work_sequences', 'eu_cellar_core',
       'eu_cellar_extended', 'metadata', 'content_path', 'content',
       'language'],
      dtype='object')

In [40]:
#What are the columns that shows dates
date_colums = [col for col in eu_cellar_df.columns if "date" in col]
eu_cellar_df[date_colums]


,dates_document,dates_created,legal_dates_entry_into_force,legal_dates_signature
_id,,,,
096cf675f70416b2e41772ca7c7e2d9e47b036fd61761f6e271d51b205d0bc50,[2020-08-24],None,None,None
098a92d90aed3b1c860da75a07011e016ffd6b932bedb844d57d4aeb722ee691,[2020-11-12],None,[2020-12-03],None
097445974db4fa8b72c8795530e3c0cfbeaa4399e1cddbfdc5a8c4e463fd6bcf,[2020-10-23],None,None,None
09a99dc1b9074fe57ab5aee6544d724ce890983416548addf972829a932708da,[2020-05-25],None,None,None
099891adb9d1f50f2ee91ca1a4a38f5acad656f8f9c48f659f6be310fb0cbfc3,[2021-04-21],None,None,None
...,...,...,...,...
7f78f989ecbe6f02cf7b35c58620f734285c0bcf930675fb33881c74935b9edb,[2021-04-14],None,None,None
0d03875f957d8d5485570408d9ed5cdd3734de99ba1a85d5ce17c328c133ec1e,[2020-10-29],None,None,None
0d5371d3179b60985fb0f25ba8954ee1807f720dc7adbdd0e9c581a8d1f120f0,[2021-03-11],None,None,None


In [41]:
# Create new dataframe with the wanted content and columns from Cellar dataset
tmp_cellar_df = pd.DataFrame(eu_cellar_df[["dates_document","title","content"]])
tmp_cellar_df["doc_source"]="eu_cellar"

In [43]:
# See the newly created Cellar dataframe
tmp_cellar_df


,Date,Title,Content,Document_source
_id,,,,
096cf675f70416b2e41772ca7c7e2d9e47b036fd61761f6e271d51b205d0bc50,[2020-08-24],Proposal for a COUNCIL IMPLEMENTING DECISION o...,IMMC.COM%282020%29470%20final.ENG.xhtml.1_EN_A...,eu_cellar
098a92d90aed3b1c860da75a07011e016ffd6b932bedb844d57d4aeb722ee691,[2020-11-12],None,,eu_cellar
097445974db4fa8b72c8795530e3c0cfbeaa4399e1cddbfdc5a8c4e463fd6bcf,[2020-10-23],None,,eu_cellar
09a99dc1b9074fe57ab5aee6544d724ce890983416548addf972829a932708da,[2020-05-25],None,,eu_cellar
099891adb9d1f50f2ee91ca1a4a38f5acad656f8f9c48f659f6be310fb0cbfc3,[2021-04-21],Proposal for a REGULATION OF THE EUROPEAN PARL...,IMMC.COM%282021%29206%20final.ENG.xhtml.1_EN_A...,eu_cellar
...,...,...,...,...
7f78f989ecbe6f02cf7b35c58620f734285c0bcf930675fb33881c74935b9edb,[2021-04-14],None,,eu_cellar
0d03875f957d8d5485570408d9ed5cdd3734de99ba1a85d5ce17c328c133ec1e,[2020-10-29],None,,eu_cellar
0d5371d3179b60985fb0f25ba8954ee1807f720dc7adbdd0e9c581a8d1f120f0,[2021-03-11],None,,eu_cellar


In [44]:
# Print dataframe columns
eu_timeline_df.columns

Index(['month_name', 'date', 'title', 'abstract', 'presscorner_links',
       'all_links', 'detail_link', 'detail_type', 'detail_date',
       'detail_location', 'detail_content', 'detail_title', 'detail_pdf_link',
       'press_contacts', 'topics', 'for_more_information_links'],
      dtype='object')

In [45]:
#What are the columns that shows dates
date_columns = [col for col in eu_timeline_df.columns if "date" in col]
eu_timeline_df[date_columns]


,date,detail_date
_id,,
01dc34ca5f6bfa5316012d6f89a4256b8d2ce6e3a12210b0406903dd15bf36ac,28 January,NaN
04d776c061243576ffc1e1c5a724d41ecca2ed12339b2ff473c5aa2ff1a071c7,22 June,22 June 2021
04eb55005a803c7353e5d5ed339d7f3a72cc8da6e49f2b8b152bf8fb40a1398f,16 June,17 June 2020
073e22ec2b44a5f2f6289f09dc954ecd6ca6cbb4a4b33ffd88c3c150572569c0,15 April,15 April 2020
0a3650bef3282d09d971dd92dc01570b7ecbdb67a4bdd0ff1c8cf8443a06b937,18 December,NaN
...,...,...
fa793c764aaf89c283f747036c9d73fd66f0a7e3692be56b8ac985610bef2417,29 June,29 June 2021
fb9c4d1634da94c73fd529b44ec40057c28a5c925a8b8e71d73c49e5a815784d,13 March,13 March 2020
fbad1d421cf80d3e0e4c6b6d3d60a2a6ec8bed4ee19e07cd15d963fa74352420,28 January,19 March 2020


In [46]:
# Create new dataframe with the wanted content and columns from EU timeline dataset
tmp_eu_timeline_df=pd.DataFrame(eu_timeline_df[["date", "title", "detail_content"]])
tmp_eu_timeline_df["doc_source"]="eu_timeline"

In [47]:
# See the newly created EU timeline  dataframe
tmp_eu_timeline_df

,date,title,detail_content,doc_source
_id,,,,
01dc34ca5f6bfa5316012d6f89a4256b8d2ce6e3a12210b0406903dd15bf36ac,28 January,EU countries adopt guidelines on proof of vacc...,NaN,eu_timeline
04d776c061243576ffc1e1c5a724d41ecca2ed12339b2ff473c5aa2ff1a071c7,22 June,Commission endorses Latvia's €1.8 billion reco...,\n\n\n\n\n\n\n\nThe European Commission has to...,eu_timeline
04eb55005a803c7353e5d5ed339d7f3a72cc8da6e49f2b8b152bf8fb40a1398f,16 June,Commission unveils EU vaccines strategy,"\n\n\n\n\n\n\n\nToday, to help protect people ...",eu_timeline
073e22ec2b44a5f2f6289f09dc954ecd6ca6cbb4a4b33ffd88c3c150572569c0,15 April,European roadmap shows path towards common lif...,"\n\n\n\n\n\n\n\nToday, the Commission, in coop...",eu_timeline
0a3650bef3282d09d971dd92dc01570b7ecbdb67a4bdd0ff1c8cf8443a06b937,18 December,Commission puts forward rules on rapid antigen...,NaN,eu_timeline
...,...,...,...,...
fa793c764aaf89c283f747036c9d73fd66f0a7e3692be56b8ac985610bef2417,29 June,Commission identifies five promising candidate...,\n\n\n\n\n\n\n\nThe EU Strategy on COVID-19 Th...,eu_timeline
fb9c4d1634da94c73fd529b44ec40057c28a5c925a8b8e71d73c49e5a815784d,13 March,Setting out coordinated response to counter th...,\n\n\n\n\n\n\n\nCOVID-19 is a severe public he...,eu_timeline
fbad1d421cf80d3e0e4c6b6d3d60a2a6ec8bed4ee19e07cd15d963fa74352420,28 January,Prolonging State aid Temporary Framework to fu...,\n\n\n\n\n\n\n\nThe European Commission has ad...,eu_timeline


In [48]:
# Print dataframe columns
ir_timeline_df.columns


Index(['keyword', 'page_type', 'page_link', 'department_data',
       'published_date', 'updated_date', 'title', 'content', 'content_links',
       'campaigns_links', 'part_of_links', 'documents'],
      dtype='object')

In [49]:
#What are the columns that shows dates
date_columns = [col for col in ir_timeline_df.columns if "date" in col]
ir_timeline_df[date_columns]



,published_date,updated_date
_id,,
003ea2a9313db86a3e0b9cd272d4dd861ed92cfab3a5d120725211cc113b1f93,17 February 2021,17 February 2021
004a77d6db5f3be729b4b3d8b7eb0ccb564defb037a0edcba96eb5c164d87a37,31 December 2020,22 April 2021
005af836e8f7b1d03afe563d254632972036a1fbd6092b7965fe8b09a73e516b,19 April 2020,19 April 2020
007344aa53333e390623e94ff899bd949ecc9f75a73937fdbcc9eebe6ffa6879,21 October 2020,21 October 2020
00ab798bb7ed96ce3b93b9a3488522a58853bcaaf1bfa1bcf05fd8ffd8af0ce1,18 December 2020,18 December 2020
...,...,...
ff02d332755586b0a04038d07386167ad32b504eeb60058e5ab9ac927ba750b3,30 October 2020,30 October 2020
ff474a2f55ba0a59240e4e986870ba12be9f2238bec4f4f8ca2c9c712172ed43,5 February 2021,8 February 2021
ff4ea2ef933287c4125d898245e878ae9fcb1b356b3c60de288dd97020bdd3a5,22 January 2021,23 January 2021


In [50]:
# Create new dataframe with the wanted content and columns from Ireland timeline dataset
tmp_ir_timeline_df=pd.DataFrame(ir_timeline_df[["published_date", "title", "content"]])
tmp_ir_timeline_df["doc_source"]="ireland_timeline"

In [51]:
# See the newly created Ireland timeline  dataframe
tmp_ir_timeline_df


,published_date,title,content,doc_source
_id,,,,
003ea2a9313db86a3e0b9cd272d4dd861ed92cfab3a5d120725211cc113b1f93,17 February 2021,Briefing on the government's response to COVID...,\n\n\n\n\n\n\n\n\n \n ...,ireland_timeline
004a77d6db5f3be729b4b3d8b7eb0ccb564defb037a0edcba96eb5c164d87a37,31 December 2020,Air passenger rights in the European Union,\n\n\n\n\n\n\n\n\n \n ...,ireland_timeline
005af836e8f7b1d03afe563d254632972036a1fbd6092b7965fe8b09a73e516b,19 April 2020,Statement from the National Public Health Emer...,\n\n\n\n\n\n\n\n\n \n ...,ireland_timeline
007344aa53333e390623e94ff899bd949ecc9f75a73937fdbcc9eebe6ffa6879,21 October 2020,Minister Hackett launches new training videos ...,\n\n\n\n\n\n\n\n\n \n ...,ireland_timeline
00ab798bb7ed96ce3b93b9a3488522a58853bcaaf1bfa1bcf05fd8ffd8af0ce1,18 December 2020,Decade of Centenaries - Minister Martin double...,\n\n\n\n\n\n\n\n\n \n ...,ireland_timeline
...,...,...,...,...
ff02d332755586b0a04038d07386167ad32b504eeb60058e5ab9ac927ba750b3,30 October 2020,Minister for Health welcomes the EU Commission...,\n\n\n\n\n\n\n\n\n \n ...,ireland_timeline
ff474a2f55ba0a59240e4e986870ba12be9f2238bec4f4f8ca2c9c712172ed43,5 February 2021,Statement from the National Public Health Emer...,\n\n\n\n\n\n\n\n\n \n ...,ireland_timeline
ff4ea2ef933287c4125d898245e878ae9fcb1b356b3c60de288dd97020bdd3a5,22 January 2021,Statement from the National Public Health Emer...,\n\n\n\n\n\n\n\n\n \n ...,ireland_timeline


In [52]:
#Renaming columns in dataframes and dropping rows without content, title and date
data_frames = [tmp_cellar_df, tmp_eu_timeline_df,tmp_ir_timeline_df,tmp_pwdb_df]
for data_frame in data_frames:
    data_frame.columns = ["Date", "Title", "Content", "Document_source"]
    data_frame.dropna(subset=['Content'], inplace=True)
    data_frame.dropna(subset=['Title'], inplace=True)
    data_frame.dropna(subset=['Date'], inplace=True)


In [53]:
#Unified datasets dataframe
unified_datasets_df = pd.DataFrame(pd.concat(data_frames))
unified_datasets_df


,Date,Title,Content,Document_source
_id,,,,
096cf675f70416b2e41772ca7c7e2d9e47b036fd61761f6e271d51b205d0bc50,[2020-08-24],Proposal for a COUNCIL IMPLEMENTING DECISION o...,IMMC.COM%282020%29470%20final.ENG.xhtml.1_EN_A...,eu_cellar
099891adb9d1f50f2ee91ca1a4a38f5acad656f8f9c48f659f6be310fb0cbfc3,[2021-04-21],Proposal for a REGULATION OF THE EUROPEAN PARL...,IMMC.COM%282021%29206%20final.ENG.xhtml.1_EN_A...,eu_cellar
0a0ff5ceccb6615dee0f62334cf70dada64b6d814113b68d43edb4b077271b7c,[2020-05-19],Proposal for a COUNCIL DECISION amending Counc...,IMMC.COM%282020%29219%20final.ENG.xhtml.2_EN_A...,eu_cellar
0ab9359aea9612ac321475299592db445c3a16d64bef6900435fe14f0f3866ca,[2020-10-26],COMMUNICATION FROM THE COMMISSION TO THE EUROP...,IMMC.COM%282020%29669%20final.ENG.xhtml.1_EN_A...,eu_cellar
0ae9ef04e65e1ea7d6effcc6f6a759eb7fa58e0e51bb4844b42aaf68b15d6c0d,[2020-10-13],Proposal for a DECISION OF THE EUROPEAN PARLIA...,IMMC.COM%282020%29960%20final%2F2.ENG.xhtml.1_...,eu_cellar
...,...,...,...,...
ff02d332755586b0a04038d07386167ad32b504eeb60058e5ab9ac927ba750b3,30 October 2020,Minister for Health welcomes the EU Commission...,\n\n\n\n\n\n\n\n\n \n ...,ireland_timeline
ff474a2f55ba0a59240e4e986870ba12be9f2238bec4f4f8ca2c9c712172ed43,5 February 2021,Statement from the National Public Health Emer...,\n\n\n\n\n\n\n\n\n \n ...,ireland_timeline
ff4ea2ef933287c4125d898245e878ae9fcb1b356b3c60de288dd97020bdd3a5,22 January 2021,Statement from the National Public Health Emer...,\n\n\n\n\n\n\n\n\n \n ...,ireland_timeline


In [ ]:
#Upload to elastic
store_registry.es_index_store().put_dataframe(index_name="ds_unified_datasets", content=unified_datasets_df)
